# 머신러닝 with Python 
## 09 앙상블 학습 배깅(Bootstrap aggregating) 실습- wine data set

##### 배깅 알고리즘을 이용하여 나이브 베이즈 모델을 활용, 와인 데이터를 분류하는 모형을 만들어본다.

책 p.267~ 

---



> 배운점1: 

> 배운점2: 

### 데이터 불러오기

In [1]:
from sklearn import datasets

In [2]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [3]:
# 데이터 살펴보기
raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
raw_wine.feature_names      # 데이터 셋 내 피처 이름들

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [5]:
raw_wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

### 피처, 타깃 데이터 지정

In [6]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [7]:
from sklearn.model_selection import train_test_split                 # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0)    # 분리. randomstate는 고정

### 데이터 표준화

In [8]:
from sklearn.preprocessing import StandardScaler         # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                             # 표준화 스케일러 지정
std_scale.fit(X_tn)                                      # 트레이닝 피처를 기준으로 표준화를 적합 시도

X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)                     # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습

In [10]:
from sklearn.naive_bayes import GaussianNB               # 가우시안 나이브 베이즈 개별 분류기
from sklearn.ensemble import BaggingClassifier            # 배깅 앙상블을 사용하여 분류

# 배깅을 활용한 분류기 생성
clf_bagging_gnb = BaggingClassifier(base_estimator = GaussianNB(), # base_estimator : 개별 학습기 입력
                          n_estimators = 10,              # n_estimator : 개별 학습기의 개수
                          random_state = 0)              # 고정

clf_bagging_gnb.fit(X_tn_std, y_tn)                               # 적합시키기

BaggingClassifier(base_estimator=GaussianNB(), random_state=0)

### 데이터 예측

In [11]:
pred_bagging_gnb = clf_bagging_gnb.predict(X_te_std)                      # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_bagging_gnb)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_bagging_gnb)        # 실제값과 예측값을 넣음
print(accuracy)

0.9555555555555556


### confusion matrix 확인

In [13]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_bagging_gnb)
print(conf_matrix)

[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [14]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_bagging_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45

